In [68]:
from __init__ import *
import Setting as _C
reload(_C)
import utils as _U
reload(_U)
import Simulation as sim
reload(sim)
import Dataloader as _D
reload(_D)

<module 'Dataloader' from 'c:\\Users\\shuai.song\\data\\Causis\\CTA_Trend\\Dataloader.py'>

In [112]:
## Multprocessing has to be used in "main" 
if __name__ == "__main__":
    dirs = ['balance_sheet', 'features', 'trading_logs', 'v_analysis']
    
    for dir in dirs:
        if _C.VERSION not in os.listdir(f'output/{dir}'):
            os.mkdir(f'output/{dir}/{_C.VERSION}')
    
    pool = Pool(8) # using 
    for i in range(len(_D.commodities.keys())):
        pool.apply_async(func=sim.wrapper_simulation, args=(list(_D.commodities.keys())[i], False, True)) # args: scom_, logger_, f_save
    
    pool.close()
    pool.join()

In [69]:
dirs = ['balance_sheet', 'features', 'trading_logs', 'v_analysis']

for dir in dirs:
    if _C.VERSION not in os.listdir(f'output/{dir}'):
        os.mkdir(f'output/{dir}/{_C.VERSION}')

sim.wrapper_simulation('rb', logger_=True, f_save=True)

======= rb:1/15 =======
Using Local 5min  Data | Local VTD: 2010-01-04--2022-08-29, BackTest VTD: 2010-01-04--2022-08-19
Using Local D     Data | Local VTD: 2010-01-04--2022-08-19, BackTest VTD: 2010-01-04--2022-08-19
[Feature and Trigger]     | 2022-09-02 13:31:11 Done | Using  30.253 seconds
[BUY Trading trace]       | 2022-09-02 13:32:23 Done | Using  72.766 seconds
[SELL Trading trace]      | 2022-09-02 13:33:35 Done | Using  71.440 seconds
[Trading Simulation]      | 2022-09-02 13:34:39 Done | Using  64.161 seconds
